In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
import seaborn as sns
from sklearn.decomposition import PCA as sklearnPCA
from sklearn.manifold import TSNE
from ipywidgets import interactive, HBox, VBox
from sklearn.cluster import KMeans

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import warnings
warnings.filterwarnings('ignore')

In [32]:
from plotly.offline import init_notebook_mode, iplot
import plotly.offline as py
import plotly.graph_objs as go
init_notebook_mode(connected=True)

In [33]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [34]:
iris = pd.read_csv("iris.csv")

X = iris.ix[:,0:4].values
y = iris.ix[:,4].values

print(np.unique(y))

['setosa' 'versicolor' 'virginica']


In [35]:
def get_cmap(n, name='hsv'):
    '''Returns a function that maps each index in 0, 1, ..., n-1 to a distinct 
    RGB color; the keyword argument name must be a standard mpl colormap name.'''
    return plt.cm.get_cmap(name, n)

In [57]:
def formatColor(color):
    return 'rgb('+str(int(color[0]*255))+','+str(int(color[1]*255))+','+str(int(color[2]*255))+')'

    
def pca(df, currentLabel,attributes,n_clusters=[7]):
    dfcopy = df.copy()
    originalMap = df.copy()
    
    sklearn_pca = sklearnPCA(n_components=2)
    X_embedded = sklearn_pca.fit_transform(X)   

    data = df
    Y_sklearn = X_embedded

    numColors = len(np.unique(y))

    cmap = get_cmap(numColors+1)
    colorList = [cmap(i) for i in range(1, numColors+1)]


    traceArr = []
    mapPoints = []
    labelArray = np.unique(y)
    for lab, col in zip(labelArray,
                        colorList):
        mapPoints.append(data[y==lab].index)

        trace1 = go.Scatter(
            x=Y_sklearn[y==lab, 0],
            y=Y_sklearn[y==lab, 1],
            mode='markers',
            name = str(lab),
            marker=dict(
                size=5,
                color=formatColor(col),
                opacity=0.8
            ),

        )
        traceArr.append(trace1)
    f = go.FigureWidget(data=traceArr, layout=go.Layout(
            margin=dict(
                l=0,
                r=0,
                b=0,
                t=0
            )
        ))
    
    t = go.FigureWidget([go.Table(
    header=dict(values=attributes,
                fill = dict(color='#C2D4FF'),
                align = ['left'] * 5),
    cells=dict(values=[originalMap[col] for col in attributes],
               fill = dict(color='#F5F8FF'),
               align = ['left'] * 5))])

    
    
    #display(VBox((slider,f,t)))
    return f

In [58]:
pca(iris,4,iris.columns.values.tolist())

FigureWidget({
    'data': [{'marker': {'color': 'rgb(7,255,0)', 'opacity': 0.8, 'size': 5},
              'mo…